# Configuration and Utilities


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from datetime import datetime
from matplotlib.dates import date2num
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_squared_error
from tensorflow import keras

import re
import math

In [ ]:
# Date parsing
def dateparse (time_in_secs):    
    return datetime.fromtimestamp(float(time_in_secs))

In [ ]:
def highlights_attacks (df, attacks, ax):
  for index, row in attacks.iterrows():
    start = datetime.fromtimestamp(row['start'])
    end = datetime.fromtimestamp(row['end'])

    period = df[(df.index > start) & (df.index <= end)].index

    ax.axvspan(start, end, facecolor='pink', edgecolor='none', alpha=.4)

In [ ]:
def clean_header (header_name, label):
    x = re.match(r"\(\'value\',\s\'(\w+)\-\S*\-\S*\'\)", header_name)
    if x == None:
        return header_name
    return x.group(1) + '-' + label


In [ ]:
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs).astype('float32'), np.array(ys).astype('float32')

In [ ]:
def return_rmse(test, predicted, model):
    rmse = math.sqrt(mean_squared_error(test, predicted))
    print("The root mean squared error for {} model is {}.".format(model, rmse))
    return rmse

In [ ]:
attacks = pd.DataFrame(
    {
        "name": ['First Attack'],
        "start" : [1619499815],
        "end" : [1619499840]
    }
)

# Fetching datasets

## Container Traffic Dataset



### Container Traffic read and clean

In [ ]:
container_traffic = pd.read_csv('/kaggle/input/chaosml/pod_traffic.csv', parse_dates=['timestamp'], date_parser=dateparse)
container_traffic.head()

In [ ]:
container_traffic_pivoted = pd.pivot_table(data=container_traffic,index=['timestamp'], columns=['pod'])
container_traffic_pivoted.head()

In [ ]:
container_traffic_flattened = pd.DataFrame(container_traffic_pivoted.to_records())
container_traffic_flattened.columns = [clean_header(hdr,'threads') for hdr in container_traffic_flattened.columns]
container_traffic_flattened.head()

In [ ]:
# may move after merge
container_traffic_flattened['attack'] = False

for index, row in attacks.iterrows():
  start = datetime.fromtimestamp(row['start'])
  end = datetime.fromtimestamp(row['end'])
  container_traffic_flattened.loc[(container_traffic_flattened['timestamp'] > start) & (container_traffic_flattened['timestamp'] <= end), 'attack'] = True

container_traffic_flattened.head()

### Container Traffic Plot

In [ ]:
indexed_container_traffic_flattened = container_traffic_flattened
indexed_container_traffic_flattened.set_index('timestamp', inplace=True)

In [ ]:
ax = indexed_container_traffic_flattened.plot()

highlights_attacks (indexed_container_traffic_flattened, attacks, ax)


## Container Satauration CPU Dataset

### Container Saturation read and clean

In [ ]:
container_saturation_cpu = pd.read_csv('/kaggle/input/chaosml/pod_saturation_cpu.csv', parse_dates=['timestamp'], date_parser=dateparse)
container_saturation_cpu.head()

In [ ]:
container_saturation_cpu_pivoted = pd.pivot_table(data=container_saturation_cpu,index=['timestamp'], columns=['pod'])
container_traffic_pivoted.head()

In [ ]:
container_saturation_cpu_flattened = pd.DataFrame(container_saturation_cpu_pivoted.to_records())
container_saturation_cpu_flattened.columns = [clean_header(hdr,'cpu') for hdr in container_saturation_cpu_flattened.columns]
container_saturation_cpu_flattened.head()

### Container Saturation CPU Plot

In [ ]:
indexed_container_saturation_cpu_flattened = container_saturation_cpu_flattened
indexed_container_saturation_cpu_flattened.set_index('timestamp', inplace=True)

In [ ]:
ax = indexed_container_saturation_cpu_flattened.plot()

highlights_attacks (indexed_container_saturation_cpu_flattened, attacks, ax)

## Container Saturation Memory dataset

In [ ]:
container_saturation_mem = pd.read_csv('/kaggle/input/chaosml/pod_saturation_memory.csv', parse_dates=['timestamp'], date_parser=dateparse)
container_saturation_mem.head()

In [ ]:
container_saturation_mem_pivoted = pd.pivot_table(data=container_saturation_mem,index=['timestamp'], columns=['pod'])
container_saturation_mem_pivoted.head()

In [ ]:
container_saturation_mem_flattened = pd.DataFrame(container_saturation_mem_pivoted.to_records())
container_saturation_mem_flattened.columns = [clean_header(hdr,'memory') for hdr in container_saturation_mem_flattened.columns]
container_saturation_mem_flattened.head()

### Container Saturation Memory plot

In [ ]:
indexed_container_saturation_mem_flattened = container_saturation_mem_flattened
indexed_container_saturation_mem_flattened.set_index('timestamp', inplace=True)

In [ ]:
ax = indexed_container_saturation_mem_flattened.plot()

highlights_attacks (indexed_container_saturation_mem_flattened, attacks, ax)


# Merge Datasets

Documentation about `merge_asof` function may be found [here](https://towardsdatascience.com/how-to-merge-not-matching-time-series-with-pandas-7993fcbce063)

In [ ]:
merged_traffic_and_cpu = pd.merge_asof (container_traffic_flattened, container_saturation_cpu_flattened, on='timestamp', tolerance=pd.Timedelta('1s'))
merged_traffic_and_cpu.head()

In [ ]:
merged_traffuc_cpu_and_mem = pd.merge_asof (merged_traffic_and_cpu, container_saturation_mem_flattened, on='timestamp', tolerance=pd.Timedelta('1s'))
merged_traffuc_cpu_and_mem.head()

In [ ]:
merged_traffuc_cpu_and_mem.describe().transpose()

## Replace Missing Values

In [ ]:
# replace with last value

colums_to_replace_with_last_value = ['cartservice-cpu', 'currencyservice-cpu', 'checkoutservice-cpu', 'adservice-cpu', 'cartservice-cpu', 'checkoutservice-cpu',
                                     'currencyservice-cpu', 'emailservice-cpu', 'frontend-cpu', 'loadgenerator-cpu', 'paymentservice-cpu', 'productcatalogservice-cpu',
                                     'recommendationservice-cpu', 'redis-cpu', 'shippingservice-cpu', 'adservice-memory','cartservice-memory', 'checkoutservice-memory',
                                     'currencyservice-memory', 'emailservice-memory', 'frontend-memory', 'loadgenerator-memory', 'paymentservice-memory',
                                     'productcatalogservice-memory', 'recommendationservice-memory', 'redis-memory', 'shippingservice-memory']

for column in colums_to_replace_with_last_value:
    merged_traffuc_cpu_and_mem[column].replace(np.nan, merged_traffuc_cpu_and_mem[column].mean(), inplace=True)

    
# replace with max value

columns_to_replace_with_max_val = []

for column in columns_to_replace_with_max_val:
    merged_traffuc_cpu_and_mem[column].replace(np.nan, merged_traffuc_cpu_and_mem[column].max(), inplace=True)

In [ ]:
merged_traffuc_cpu_and_mem.isnull().sum(axis = 0)

### Merge Plot

In [ ]:
indexed_merge = merged_traffuc_cpu_and_mem
indexed_merge.set_index('timestamp', inplace=True)

In [ ]:
indexed_merge.plot(subplots=True, y=['frontend-threads', 'frontend-cpu', 'frontend-memory'])

# Train a model

In [ ]:
f_predicted = 'frontend-cpu' #'frontend-memory'

## Pre processing

We’ll use the last 10% of the data for testing:

In [ ]:
train_size = int(len(indexed_merge) * 0.9)
test_size = len(indexed_merge) - train_size

train, test = indexed_merge.iloc[0:train_size], indexed_merge.iloc[train_size:len(indexed_merge)]

print(len(train), len(test))

We’ll scale some of the features we’re using for our modeling:

In [ ]:
f_columns = indexed_merge.columns.to_list()
f_columns.remove('attack')
f_columns.remove(f_predicted)

f_transformer = RobustScaler()
frontend_memory_transformer = RobustScaler()

f_transformer = f_transformer.fit(train[f_columns].to_numpy())
frontend_memory_transformer = frontend_memory_transformer.fit(train[[f_predicted]])

train.loc[:, f_columns] = f_transformer.transform(
  train[f_columns].to_numpy()
)
train[f_predicted] = frontend_memory_transformer.transform(train[[f_predicted]])

test.loc[:, f_columns] = f_transformer.transform(
  test[f_columns].to_numpy()
)
test[f_predicted] = frontend_memory_transformer.transform(test[[f_predicted]])

print(len(train), len(test))
print(train.shape, test.shape)

In [ ]:
time_steps = 10

# reshape to [samples, time_steps, n_features]

X_train, y_train = create_dataset(train, train[f_predicted], time_steps)
X_test, y_test = create_dataset(test, test[f_predicted], time_steps)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

## Prediction with LSTM

In [ ]:
model_lstm = keras.Sequential()

# First LSTM layer with Dropout regularisation
model_lstm.add(keras.layers.LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model_lstm.add(keras.layers.Dropout(0.2))
# Second LSTM layer
model_lstm.add(
  keras.layers.LSTM(
    units=128,
    input_shape=(X_train.shape[1], X_train.shape[2])
  )
)
model_lstm.add(keras.layers.Dropout(rate=0.2))
# The output layer
model_lstm.add(keras.layers.Dense(units=1))

model_lstm.compile(optimizer='rmsprop',loss='mean_squared_error')

In [ ]:
history_lstm = model_lstm.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.1,
    shuffle=False
)

In [ ]:
model_lstm.summary()

In [ ]:
y_pred_lstm = model_lstm.predict(X_test)
y_pred_lstm.shape

In [ ]:
y_train_inv_lstm = frontend_memory_transformer.inverse_transform(y_train.reshape(1, -1))
y_test_inv_lstm = frontend_memory_transformer.inverse_transform(y_test.reshape(1, -1))
y_pred_inv_lstm = frontend_memory_transformer.inverse_transform(y_pred_lstm)

In [ ]:
plt.plot(np.arange(0, len(y_train)), y_train_inv_lstm.flatten(), 'g', label="history")
plt.plot(np.arange(len(y_train), len(y_train) + len(y_test)), y_test_inv_lstm.flatten(), label="true")
plt.plot(np.arange(len(y_train), len(y_train) + len(y_test)), y_pred_inv_lstm.flatten(), 'r', label="prediction")
plt.ylabel('FrontEnd CPU')
plt.xlabel('Time Step')
plt.title('LSTM')
plt.legend()
plt.show();

In [ ]:
rmse_lstm = return_rmse(y_test,y_pred_inv_lstm,'LSTM')

## Prediction with Bidirectional LSTM with a Dropout layer

In [ ]:
model_lstm_bidirectional = keras.Sequential()

# First LSTM layer with Dropout regularisation
model_lstm_bidirectional.add(keras.layers.LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model_lstm_bidirectional.add(keras.layers.Dropout(0.2))
# Second LSTM layer
model_lstm_bidirectional.add(
  keras.layers.Bidirectional(
    keras.layers.LSTM(
      units=50,
      input_shape=(X_train.shape[1], X_train.shape[2])
    )
  )
)
model_lstm_bidirectional.add(keras.layers.Dropout(rate=0.2))
# The output layer
model_lstm_bidirectional.add(keras.layers.Dense(units=1))

model_lstm_bidirectional.compile(optimizer='rmsprop',loss='mean_squared_error')

In [ ]:
history_1 = model_lstm_bidirectional.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.1,
    shuffle=False
)

In [ ]:
model_lstm_bidirectional.summary()

In [ ]:
y_pred = model_lstm_bidirectional.predict(X_test)
y_pred.shape

Reverse scaling

In [ ]:
y_train_inv = frontend_memory_transformer.inverse_transform(y_train.reshape(1, -1))
y_test_inv = frontend_memory_transformer.inverse_transform(y_test.reshape(1, -1))
y_pred_inv = frontend_memory_transformer.inverse_transform(y_pred)

In [ ]:
plt.plot(np.arange(0, len(y_train)), y_train_inv.flatten(), 'g', label="history")
plt.plot(np.arange(len(y_train), len(y_train) + len(y_test)), y_test_inv.flatten(), label="true")
plt.plot(np.arange(len(y_train), len(y_train) + len(y_test)), y_pred_inv.flatten(), 'r', label="prediction")
plt.ylabel('FrontEnd CPU')
plt.xlabel('Time Step')
plt.title('Bidirectional LSTM')
plt.legend()
plt.show();

In [ ]:
rmse_bi_lstm = return_rmse(y_test,y_pred_inv,'Bidirectional LSTM')

## Prediction with GRU

In [ ]:
model_gru = keras.Sequential()

# First GRU layer with Dropout regularisation
model_gru.add(keras.layers.GRU(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2]), activation='tanh'))
model_gru.add(keras.layers.Dropout(0.2))
# Second LSTM layer
model_gru.add(
  keras.layers.GRU(
    units=128,
    input_shape=(X_train.shape[1], X_train.shape[2]),
    activation='tanh'
  )
)
model_gru.add(keras.layers.Dropout(rate=0.2))
# The output layer
model_gru.add(keras.layers.Dense(units=1))

model_gru.compile(optimizer=keras.optimizers.SGD(lr=0.01, decay=1e-7, momentum=0.9, nesterov=False), loss='mean_squared_error')

In [ ]:
history_gru = model_lstm_bidirectional.fit(
    X_train, y_train,
    epochs=50,
    batch_size=150,
    shuffle=False
)

In [ ]:
model_gru.summary()

In [ ]:
y_pred_gru = model_gru.predict(X_test)
y_pred_gru.shape

In [ ]:
y_train_inv_gru = frontend_memory_transformer.inverse_transform(y_train.reshape(1, -1))
y_test_inv_gru = frontend_memory_transformer.inverse_transform(y_test.reshape(1, -1))
y_pred_inv_gru = frontend_memory_transformer.inverse_transform(y_pred_gru)

In [ ]:
plt.plot(np.arange(0, len(y_train)), y_train_inv_gru.flatten(), 'g', label="history")
plt.plot(np.arange(len(y_train), len(y_train) + len(y_test)), y_test_inv_gru.flatten(), label="true")
plt.plot(np.arange(len(y_train), len(y_train) + len(y_test)), y_pred_inv_gru.flatten(), 'r', label="prediction")
plt.ylabel('FrontEnd CPU')
plt.xlabel('Time Step')
plt.title('GRU')
plt.legend()
plt.show();

In [ ]:
rmse_gru = return_rmse(y_test,y_pred_inv_gru,'GRU')

## Compare Performance

In [ ]:
rmse_df = pd.DataFrame({'Model':['LSTM', 'Bidirectional LSTM', 'GRU'], 'RMSE':[rmse_lstm, rmse_bi_lstm, rmse_gru]})
ax = rmse_df.plot.bar(x='Model', y='RMSE', rot=0, color = ['green','blue', 'orange'])

for p in ax.patches:
    b = p.get_bbox()
    val = "{:.2f}".format(b.y1 + b.y0)        
    ax.annotate(val, ((b.x0 + b.x1)/2 - 0.07, b.y1 + 0.005))